# Introduction

Lets start our pipeline by making a data cleaning to the data collected from mercadolibre API (MELI), and some tasks relative to **data wrangling**. Some ot those tasks are:
- convert prices to a unify currency like(usd)
- extract useful info from columns like "tags"

In [1]:
import sys
sys.path.insert(0,'../..')

In [2]:
# Loading relevant libraries
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from scipy.stats import norm
import seaborn as sns
from utils import *

### Loading the raw data

In [3]:
raw_data = pd.read_csv('../../data/01_raw/meli_data.csv',index_col=0)
raw_data.drop(columns='order_backend',inplace=True)
# Lets take a look of some records of raw data
raw_data.sample(3)

,post_id,user_id,country,city,category_name,product,price,original_price,available_quantity,sold_quantity,condition,accepts_mercadopago,shipping_state,tags
30638,MLA736092705,14500081,Argentina,Paternal,Agro,Soldadora Ac/dc Inverter Tig Pulso 200a Envio ...,85888.0,NaN,1,250,new,True,True,"['brand_verified', 'good_quality_picture', 'go..."
2481,MCO573996561,66208468,Colombia,Martires,Celulares y Teléfonos,Radio Walkie Talkie Baofeng Bf-888s Uhf X2 Bat...,102400.0,NaN,1,150,new,True,True,"['good_quality_picture', 'good_quality_thumbna..."
5143,MCO513177503,356452965,Colombia,Medellín,"Electrónica, Audio y Video",Radio Parlante Inalámbrico Am/ Fm/ Sw Sonivox ...,55000.0,NaN,1,500,new,True,False,"['good_quality_picture', 'good_quality_thumbna..."


In [4]:
# shape of datset
print('Shape of raw_data:\n',raw_data.shape)

Shape of raw_data:
 (52265, 14)


In [5]:
# categories present in the dataset
raw_data['category_name'].value_counts()

Accesorios para Vehículos     5634
Deportes y Fitness            4648
Juegos y Juguetes             4415
Libros, Revistas y Comics     4332
Agro                          4034
Computación                   3549
Servicios                     3500
Instrumentos Musicales        3200
Electrónica, Audio y Video    2998
Inmuebles                     2950
Otras categorías              2599
Consolas y Videojuegos        2449
Autos, Motos y Otros          2400
Celulares y Teléfonos         2094
Joyas y Relojes               1650
Música y Películas            1649
Arte y Antigüedades            164
Name: category_name, dtype: int64

In [6]:
# For the category "Servicios" some of the fields have no real sense like:
# sold_quantity,avalaible_quantity,original_price,condition.

raw_data[raw_data['category_name']=='Servicios'].head(3)

,post_id,user_id,country,city,category_name,product,price,original_price,available_quantity,sold_quantity,condition,accepts_mercadopago,shipping_state,tags
8698,MCO580150997,22300563,Colombia,Madrid,Servicios,Alquiler De Vans Sin Conductor,400000.0,NaN,1,0,NaN,False,False,['dragged_visits']
8699,MCO580152287,22300563,Colombia,Madrid,Servicios,Alquiler De Vehículos Al Mejor Precio,125000.0,NaN,1,0,NaN,False,False,['dragged_visits']
8700,MCO574555437,236862201,Colombia,Ciudad Bolivar,Servicios,Cursos Licencias De Conducción A2 Moto B1 - ...,NaN,NaN,1,0,NaN,False,False,['dragged_visits']


In [7]:
# discard the records with Servicio category.
raw_datav2 = raw_data[raw_data['category_name']!='Servicios']
raw_datav2
print('Shape of data without servicios category:\n',raw_datav2.shape)

Shape of data without servicios category:
 (48765, 14)


## Data preprocessing

Lets make some little changes in a couple of columns to get a better and useful dataset:
- handle missing values
- unify prices currency to usd
- coding tag field
- handle the product name field

In [8]:
# checking the types of data we have
raw_datav2.dtypes

post_id                 object
user_id                  int64
country                 object
city                    object
category_name           object
product                 object
price                  float64
original_price         float64
available_quantity       int64
sold_quantity            int64
condition               object
accepts_mercadopago       bool
shipping_state            bool
tags                    object
dtype: object

In [9]:
#duplicated records
raw_datav2.iloc[:,:].duplicated().value_counts()

False    48743
True        22
dtype: int64

In [10]:
#raw_datav2[raw_datav2.duplicated()]

### handling missing values

In [11]:
# lets see whats columns have missing or null values:
# - price
#- original_price

raw_datav3 = raw_datav2.copy(deep=True)
raw_datav3.isna().any()

post_id                False
user_id                False
country                False
city                   False
category_name          False
product                False
price                   True
original_price          True
available_quantity     False
sold_quantity          False
condition              False
accepts_mercadopago    False
shipping_state         False
tags                   False
dtype: bool

In [12]:
# complete the cases with original_price null with the current price field 
indx = raw_datav3[raw_datav3['original_price'].isna()==True].index
# fill na with the current price value
raw_datav3.loc[indx,'original_price']=raw_datav3.loc[indx,'price']

In [13]:
# missing values in the field "price".
print('records with na in price field: ',raw_datav3[raw_datav3['price'].isna()==True].shape[0])
raw_datav3[raw_datav3['price'].isna()==True].head(3)

records with na in price field:  14


,post_id,user_id,country,city,category_name,product,price,original_price,available_quantity,sold_quantity,condition,accepts_mercadopago,shipping_state,tags
43947,MLV572718272,38480507,Venezuela,chacao,Inmuebles,Lomas De Chuao Extraordinaria Propiedad Con Vista,NaN,NaN,1,0,used,False,False,"['dragged_visits', 'good_quality_picture']"
43948,MLV572718485,38480507,Venezuela,chacao,Inmuebles,Bello Apartamento Listo Para Entrar San Bernar...,NaN,NaN,1,0,used,False,False,"['dragged_visits', 'good_quality_picture']"
43949,MLV570612245,38480507,Venezuela,chacao,Inmuebles,Bella Casa En Exclusivo Desarrollo En Caurimare,NaN,NaN,1,0,used,False,False,['good_quality_picture']


In [14]:
# Finally take off all the 14 cases with a null in price field.
raw_datav3.dropna(inplace=True)
print('Shape the cleaner version of dataset:\n',raw_datav3.shape)

Shape the cleaner version of dataset:
 (48751, 14)


In [15]:
# reset index for the dataset
raw_datav3.reset_index(drop=True,inplace=True)

### Change prices to usd

In [17]:
import json

with open('../../data/01_raw/currency_ids.json') as f:
    currency_ids = json.load(f)

In [18]:
raw_datav3['price'] = currency_converter(currency_ids,raw_datav3['country'],raw_datav3['price'])
raw_datav3['original_price']=currency_converter(currency_ids,raw_datav3['country'],raw_datav3['original_price'])
raw_datav3.sample(3)

,post_id,user_id,country,city,category_name,product,price,original_price,available_quantity,sold_quantity,condition,accepts_mercadopago,shipping_state,tags
13945,MLU471702182,104702637,Uruguay,Cordón,"Libros, Revistas y Comics","Gago, Soledad - Uruguayas Rebeldes",490.000000,490.000000,1,4,new,True,False,"['poor_quality_picture', 'immediate_payment', ..."
4764,MCO576995834,29789735,Colombia,Santa Fe,Deportes y Fitness,Banda De Poder Elástica Morada Sportfitness 50...,11.922652,16.081717,1,100,new,True,False,"['good_quality_picture', 'good_quality_thumbna..."
23494,MLC514137584,91518386,Chile,Conchalí,Deportes y Fitness,Pack 10 Saleros Molinillos Negros Sal + 10 Sel...,15.678175,15.678175,150,500,new,True,False,"['good_quality_picture', 'good_quality_thumbna..."


### Short product name extraction

we are gonna create a short version of the product name taking into account that usually the first part of the product name give a relative good description.

In [19]:
raw_datav3['product'] = product_name_extractor(raw_datav3['product'])

In [20]:
raw_datav3.head(3)

,post_id,user_id,country,city,category_name,product,price,original_price,available_quantity,sold_quantity,condition,accepts_mercadopago,shipping_state,tags
0,MCO548989931,145181776,Colombia,Medellín,Accesorios para Vehículos,Espejo,3.049981,3.049981,50,500,new,True,False,"['good_quality_picture', 'good_quality_thumbna..."
1,MCO508601525,152737872,Colombia,Kennedy,Accesorios para Vehículos,Forro,11.063112,11.063112,150,500,new,True,True,"['good_quality_picture', 'good_quality_thumbna..."
2,MCO509093160,212962423,Colombia,Cali,Accesorios para Vehículos,Radio,59.585535,59.585535,250,500,new,True,True,"['good_quality_picture', 'good_quality_thumbna..."


In [21]:
# Get the stacked list of tags and the tags freq dict
stacked_tags,tags_freq = tags_extractor(raw_datav3['tags'])

In [22]:
raw_datav4= raw_datav3.copy(deep=True)

In [23]:
start = time.time()
data_tag_filled = add_tags_fields(raw_datav4,stacked_tags,tags_freq)
end= time.time()
print('Execution time in seconds: ', round(end-start,2))

Execution time in seconds:  106.33


In [24]:
# take a copy of the cleaner data before create the discount percentage field
data = data_tag_filled.copy(deep=True)

### Discount percentage Column

Some products exhibit some "*discount*" in their price, so from the customer point of view the knowledge of the "*discount percentage*" could be important to encourage  to put more attention to the offer. 

In [25]:
data['discount_percentage'] = np.round((1- data['price']/data['original_price'])*100, decimals=2)

In [26]:
columns = ['post_id','user_id','country','city','category_name','product','available_quantity', 'sold_quantity',
       'good_quality_picture', 'immediate_payment', 'good_quality_thumbnail',
       'cart_eligible', 'shipping_guaranteed', 'dragged_bids_and_visits',
       'brand_verified', 'loyalty_discount_eligible', 'poor_quality_picture',
       'best_seller_candidate', 'incomplete_technical_specs', 'dragged_visits',
       'poor_quality_thumbnail', 'extended_warranty_eligible', 'free_relist',
       'ahora', 'catalog_product_candidate', 'only_html_description',
       'catalog_listing_eligible', 'cbt_item', 'under_infractions',
       '3x_campaign', 'deal_of_the_day', 'unknown_quality_picture',
       'catalog_boost']

convert_to =['category']

# change the dtype of a list of columns
data = convert_column_dtype(data,columns,convert_to)

In [59]:
# save the file
#data.to_csv(r'.\data\02_intermediate\meli_data_test.csv')